In [1]:
# 导入模块
import os # 用于操作系统的交互，例如获取环境变量。
import openai # OpenAI 的官方库，用于与 OpenAI 服务进行交互。
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY'] # 从环境变量中获取 OPENAI_API_KEY 作为 API 密钥。
openai.api_type = "azure"
openai.api_base = "https://zjx.openai.azure.com/"
openai.api_version = "2023-05-15"

In [2]:
# 定义了一个名为 get_completion 的函数，其主要目的是使用 OpenAI 的 API 为给定的 prompt 获取模型的回复
def get_completion(prompt):
    # messages 是一个列表，其中包含一个字典，这个字典定义了一个用户角色和他们的消息内容。
    # 这是 OpenAI 的 Chat API 接受的格式，它允许多次交互（例如，先由用户发送消息，然后由模型回复，然后再由用户发送消息，等等）
    messages = [{"role": "user", "content": prompt}] 
    # 使用 try 和 except 来尝试执行某些代码，并在出现异常时捕获该异常。
    try:
        response = openai.ChatCompletion.create(
            engine="gpt35",  # 指定使用哪个模型。在这里，它是 "gpt35"。
            messages=messages,
            temperature=0, # 控制输出随机性的参数。值为 0 表示模型会产生最确定性的输出；较高的值（例如 1.0）会产生更多的随机性。
        )
        return response.choices[0].message["content"] # 从模型的响应中获取其消息内容并返回。
    except Exception as e:
        print(f"Error: {e}")
        return None

In [3]:
## 在这里，max_tokens主要限制的是输出的令牌数量，也就是模型生成的内容的最大长度。
def get_completion_from_messages(messages, 
                                 max_tokens=500):
    try:
        response = openai.ChatCompletion.create(
            max_tokens=max_tokens,
            engine="gpt35",
            messages=messages,
            temperature=0,
        )
    except Exception as e:
        print(f"Error: {e}")
    return response.choices[0].message["content"]

In [4]:
# 输入处理： 思考链推理
delimiter = "####"
system_message = f"""
按照以下步骤回答客户的问题。
客户的问题将用四个井号分隔，即 {delimiter}。

步骤1:{delimiter} 首先判断用户是否询问有关特定产品或多个产品的问题。
仅提及产品类别不算。

步骤2:{delimiter} 如果用户询问特定的产品，确定这些产品是否在下面的列表中。
所有可用的产品： 
1. 产品：TechPro 超极本
   类别：计算机和笔记本
   品牌：TechPro
   型号：TP-UB100
   保修：1年
   评分：4.5
   功能：13.3英寸显示屏，8GB RAM，256GB SSD，Intel Core i5处理器
   描述：适合日常使用的时尚轻薄超极本。
   价格：$799.99

2. 产品：BlueWave 游戏笔记本
   类别：计算机和笔记本
   品牌：BlueWave
   型号：BW-GL200
   保修：2年
   评分：4.7
   功能：15.6英寸显示屏，16GB RAM，512GB SSD，NVIDIA GeForce RTX 3060
   描述：为沉浸式体验提供的高性能游戏笔记本。
   价格：$1199.99

3. 产品：PowerLite 二合一笔记本
   类别：计算机和笔记本
   品牌：PowerLite
   型号：PL-CV300
   保修：1年
   评分：4.3
   功能：14英寸触摸屏，8GB RAM，256GB SSD，360度转轴
   描述：具有响应式触摸屏的多功能二合一笔记本。
   价格：$699.99

4. 产品：TechPro 台式机
   类别：计算机和笔记本
   品牌：TechPro
   型号：TP-DT500
   保修：1年
   评分：4.4
   功能：Intel Core i7处理器，16GB RAM，1TB HDD，NVIDIA GeForce GTX 1660
   描述：适合工作和娱乐的强大台式机。
   价格：$999.99

5. 产品：BlueWave Chromebook
   类别：计算机和笔记本
   品牌：BlueWave
   型号：BW-CB100
   保修：1年
   评分：4.1
   功能：11.6英寸显示屏，4GB RAM，32GB eMMC，Chrome OS
   描述：适合日常任务的紧凑且实惠的Chromebook。
   价格：$249.99

步骤3:{delimiter} 如果消息中包含上述列表中的产品，列出用户在消息中可能的假设，
例如笔记本X比笔记本Y大，或笔记本Z有2年保修期。

步骤4:{delimiter} 如果用户做了任何假设，根据您的产品信息确定这些假设是否正确。

步骤5:{delimiter} 首先，如果适用的话，礼貌地纠正客户的错误假设。
只提及或参考上述5种可用产品中的产品，因为这些是商店唯一销售的5种产品。
用友好的语气回答客户。

请使用以下格式：
步骤1:{delimiter} <步骤1的推理>
步骤2:{delimiter} <步骤2的推理>
步骤3:{delimiter} <步骤3的推理>
步骤4:{delimiter} <步骤4的推理>
回复用户:{delimiter} <回复客户的内容>

确保包括 {delimiter} 来分隔每一步。
"""


In [5]:
user_message = f"""
BlueWave Chromebook比TechPro台式机贵多少？
"""

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 

response = get_completion_from_messages(messages)
print(response)


步骤1:#### 这是有关特定产品的问题。

步骤2:#### BlueWave Chromebook和TechPro台式机都在我们的产品列表中。

步骤3:#### 用户假设BlueWave Chromebook比TechPro台式机贵。

步骤4:#### 这个假设是不正确的。TechPro台式机的价格是$999.99，而BlueWave Chromebook的价格是$249.99，因此TechPro台式机比BlueWave Chromebook贵了$750。

回复用户:#### TechPro台式机比BlueWave Chromebook贵了$750。
